# Fluidinámica computacional (Sesión 06)

<img src="figures/LogoCinvestav.png" style="height:100px" title="Cinvestav">

**CINVESTAV Unidad Saltillo**

**Programa de Maestría en Ciencias en Ingeniería Metalúrgica**

Dr. Edgar Ivan Castro Cedeño


Contacto:
[edgar.castro@cinvestav.mx](mailto:edgar.castro@cinvestav.mx)

# Acoplamiento presión - velocidad

## Fundamentos

**Conservación de cantidad de movimiento** (resolver para $\mathbf{u}$):

$$
\frac{\partial \mathbf{u}}{\partial t} + \mathbf{\nabla \cdot} (\mathbf{u u}) - \mathbf{\nabla \cdot} (\nu \mathbf{\nabla u}) - \frac{1}{\rho} \mathbf{b}
=
- \frac{1}{\rho} \mathbf{\nabla} p
$$



**Conservación de masa** (resolver para $p$):

$$
\mathbf{\nabla \cdot u} = 0 \quad \rightarrow \quad \frac{1}{\rho} \mathbf{\nabla^2}p + \mathbf{\nabla \cdot} \left[\mathbf{\nabla \cdot} (\mathbf{u u})\right] = 0
$$

<details>
<summary><b>Nomenclatura</b></summary>

<div class="alert alert-info">

**$t$**: tiempo, en $[\mathrm{s}]$.

**$\mathbf{u}$**: vector de velocidad, $\mathbf{u} = \begin{pmatrix}
	u_x & u_y & u_z
	\end{pmatrix}$, en $[\mathrm{m.s^{-1}}]$.

**$\nu$**: viscosidad cinemática, en $[\mathrm{m^{2}.s}]$.

**$p$**: presión, en $[\mathrm{Pa}]$ (**SI:** $[\mathrm{kg.m^{-1}.s^{-2}}]$).

**$\rho$**: densidad, en $[\mathrm{kg.m^{-3}}]$
	
**$\mathbf{b}$:** Término fuente, $\mathbf{b} = \begin{pmatrix} b_x & b_y & b_z \end{pmatrix}$, en $[\mathrm{N.m^{-3}}]$ (**SI:** $[\mathrm{kg.m^{-2}.s^{-2}}]$).

**$\nabla:$** Operador nabla, $\nabla = \begin{pmatrix} \frac{\partial}{\partial x} & \frac{\partial}{\partial y} & \frac{\partial}{\partial z} \end{pmatrix}$, en $[\mathrm{m^{-1}}]$.

</div>

</details>

### Ecuación de cantidad de movimiento (sin término de presión)

Los algoritmos utilizados para acoplar presión y velocidad, de una forma que se logre la convergencia numérica del sistema, utilizan la siguiente notación para describir términos en la ecuación de cantidad de movimiento (excluyendo el término $\mathbf{\nabla}p$).


$$
\mathbf{A u} - \mathbf{b}
\equiv
\frac{\partial \mathbf{u}}{\partial t} + \mathbf{\nabla \cdot} (\mathbf{u u}) - \mathbf{\nabla \cdot} (\nu \mathbf{\nabla u}) - \frac{1}{\rho} \mathbf{b}
$$

<details>
<summary><b>Descomposición</b></summary>

<div class="alert alert-success">

$$
\underbrace{
\begin{bmatrix}
\square & * & & * \\ * & \square & * & \\ & * & \square & * \\ * & & * & \square
\end{bmatrix}
\begin{bmatrix} \\ \mathbf{u} \\ \\ \end{bmatrix}
}_{\mathbf{A u}}
-
\underbrace{
\begin{bmatrix} * \\ * \\ * \\ * \end{bmatrix}
}_{\mathbf{b}}
$$

</div>

</details>

$$
A\mathbf{u} - \mathbf{H}(\mathbf{u})
\equiv
\frac{\partial \mathbf{u}}{\partial t} + \mathbf{\nabla \cdot} (\mathbf{u u}) - \mathbf{\nabla \cdot} (\nu \mathbf{\nabla u}) - \frac{1}{\rho} \mathbf{b}
$$

<details>
<summary><b>Descomposición</b></summary>

<div class="alert alert-success">

$$
	\underbrace{
	\begin{bmatrix}
	\square &  & &  \\  & \square &  & \\ &  & \square &  \\  & &  & \square
	\end{bmatrix}
	\begin{bmatrix} \\ \mathbf{u} \\ \\ \end{bmatrix}
	}_{A\mathbf{u}}
	+
	\underbrace{
	\begin{bmatrix}
	 & * & & * \\ * &  & * & \\ & * &  & * \\ * & & * & 
	\end{bmatrix}
	\begin{bmatrix} \\ \mathbf{u} \\ \\ \end{bmatrix}
	-
	\begin{bmatrix} * \\ * \\ * \\ * \end{bmatrix}
	}_{-\mathbf{H(u)}}
$$

</div>

</details>

<details>
<summary><b>Nomenclatura</b></summary>

<div class="alert alert-info">

**$A \mathbf{u}$**: término lineal de $\mathbf{u}$.

**$-\mathbf{H(u)}$**: una función de $\mathbf{u}$ y otras fuentes.

</div>

</details>

### Corrección de momentum

La ecuación de cantidad de movimiento se puede expresar en términos de la ecuación presentada arriba.

$$
\frac{\partial \mathbf{u}}{\partial t} + \mathbf{\nabla \cdot} (\mathbf{u u}) - \mathbf{\nabla \cdot} (\nu \mathbf{\nabla u}) + \frac{1}{\rho} \mathbf{b} = - \frac{1}{\rho} \mathbf{\nabla} p 
$$

$$
A\mathbf{u} - \mathbf{H}(\mathbf{u}) = - \frac{1}{\rho} \mathbf{\nabla} p
$$

Reacomodando la ecuación, se obtiene una expresión que permite actualizar la velocidad $\mathbf{u}$, tomando en cuenta los valores iterados de $\mathbf{u}, p$.

$$
\mathbf{u} := \frac{\mathbf{H(u)}}{A} - \frac{1}{\rho} \frac{1}{A} \mathbf{\nabla} p
$$

### Corrección de flujo

A partir de la ecuación de corrección de momentum se deriva una ecuación de corrección de flujo, al interpolar $\mathbf{u}$ a las caras del volumen de control, y evaluando los flujos en las caras.

$$
\phi_f = \mathbf{S}_f \cdot \mathbf{u}_f
$$

$$
\phi_f := \mathbf{S}_f \cdot \left(\frac{\mathbf{H(u)}}{A} \right) - \frac{1}{\rho} \left(\frac{|\mathbf{S}_f|}{A}\right) \mathbf{\nabla_n} (p_f)
$$

### Ecuación de presión

Tras sustituir los flujos $\phi_f$ en la ecuación de conservación discretizada, $\mathbf{\nabla \cdot u} \approx \sum_f \phi_f = 0$, se obtiene una expresión para la ecuación de presión discretizada, con coeficientes que contienen $A$ y $\mathbf{H(u)}$.

$$
\frac{1}{\rho} \mathbf{\nabla \cdot} \frac{1}{A} \mathbf{\nabla} p = \mathbf{\nabla \cdot}\left[\frac{\mathbf{H}(\mathbf{u})}{A}\right]
$$

<details>
<summary><b>Derivación</b></summary>

<div class="alert alert-success">

La ecuación de cantidad de movimiento

$$
A\mathbf{u} - \mathbf{H}(\mathbf{u}) = - \frac{1}{\rho} \mathbf{\nabla} p
$$

se reescribe como:

$$
\mathbf{u} - \left[\frac{\mathbf{H}(\mathbf{u})}{A}\right] = - \frac{1}{\rho} \frac{1}{A} \mathbf{\nabla} p
$$

Tomando la divergencia:
$$
\cancel{\mathbf{\nabla \cdot u}} - \mathbf{\nabla \cdot}\left[\frac{\mathbf{H}(\mathbf{u})}{A}\right] = - \frac{1}{\rho} \mathbf{\nabla \cdot} \frac{1}{A} \mathbf{\nabla} p
$$

$$
\cancel{\sum_f \phi_f} - \mathbf{\nabla \cdot}\left[\frac{\mathbf{H}(\mathbf{u})}{A}\right] = - \frac{1}{\rho} \mathbf{\nabla \cdot} \frac{1}{A} \mathbf{\nabla} p
$$

Reacomodando:
$$
\frac{1}{\rho} \mathbf{\nabla \cdot} \frac{1}{A} \mathbf{\nabla} p = \mathbf{\nabla \cdot}\left[\frac{\mathbf{H}(\mathbf{u})}{A}\right]
$$

</div>

</details>

## Algoritmo SIMPLE

**SIMPLE (Semi-Implicit Pressure Linking Equations, 1972)**

1. Se construye una ecuación matricial usando los términos de la ecuación de momentum, excluyendo el término de presión, $\nabla p$ **(Momentum matrix)**.

2. Se relaja la ecuación matricial resultante con un factor $\alpha_u$.

3. La ecuación matricial relajada se iguala al término de presión, $\nabla p$, y se resuelve para la velocidad, $\mathbf{u}$ **(Momentum predictor)**.

4. Se evalúan $A\mathbf{u}$ y $\mathbf{H(u)}$ a partir de $\mathbf{Au-b}$ (la matriz de momentum). Se utilizan para construir la ecuación de presión, que se resuelve para $p$ **(Pressure equation)**.

5. La nueva presión $p$ se utiliza para corregir los flujos en las caras $\phi_f$, para lograr una mejor conservación de masa **(Flux corrector)**.

6. Se relaja la presión con un factor $\alpha_p$.

7. Se corrige la velocidad $\mathbf{u}$ previo a proceder con el siguiente paso de la solución **(Momentum corrector)**.

<center>

<img src="figures/images/simpleMod.png" style="width:400px" title="Geometría 1D">

***Figura 01. Esquema del algoritmo SIMPLE.***

</center>

El término de presión expresando en la figura corresponde a una "presión dinámica", $\hat{p} = p/\rho$.

Adaptado de: [[CFD Direct, 2023]](https://doc.cfd.direct/notes/cfd-general-principles/numerical-method)

## Algoritmo PISO

**PISO (Pressure Implicit with Splitting of Operators, 1986)**

1. Se construye una ecuación matricial usando los términos de la ecuación de momentum, excluyendo el término de presión, $\nabla p$ **(Momentum matrix)**.

2. La ecuación matricial se iguala al término de presión, $\nabla p$ **(Momentum predictor)**.

4. Se evalúan $A\mathbf{u}$ y $\mathbf{H(u)}$ a partir de $\mathbf{Au-b}$ (la matriz de momentum). Se utilizan para construir la ecuación de presión, que se resuelve para $p$ **(Pressure equation)**.

5. La nueva presión $p$ se utiliza para corregir la velocidad, $\mathbf{u}$ **(Momentum corrector)**.

6. Se vuelve a corregir de nuevo la presión, y esto es seguido de una nueva corrección de momentum **(PISO loop)**.

7. Se actualiza el término advectivo, i.e, los flujos en las caras $\phi_f$ antes de pasar al paso de tiempo posterior **(Flux corrector)**.

<center>

<img src="figures/images/pisoMod.png" style="width:400px" title="Geometría 1D">

***Figura 02. Esquema del algoritmo PISO.***

</center>

El término de presión expresando en la figura corresponde a una "presión dinámica", $\hat{p} = p/\rho$.

Adaptado de: [[CFD Direct, 2023]](https://doc.cfd.direct/notes/cfd-general-principles/numerical-method)

## Algoritmo PIMPLE (OpenFoam)

**PIMPLE (PISO + SIMPLE, ~1996)**

1. Se construye una ecuación matricial usando los términos de la ecuación de momentum, excluyendo el término de presión, $\nabla p$ **(Momentum matrix)**.

2. Se relaja la ecuación matricial resultante con un factor $\alpha_u$.

3. La ecuación matricial relajada se iguala al término de presión, $\nabla p$, y se resuelve para la velocidad, $\mathbf{u}$ **(Momentum predictor)**.

4. Se evalúan $A\mathbf{u}$ y $\mathbf{H(u)}$ a partir de $\mathbf{Au-b}$ (la matriz de momentum). Se utilizan para construir la ecuación de presión, que se resuelve para $p$ **(Pressure equation)**.

5. La nueva presión $p$ se utiliza para corregir los flujos en las caras $\phi_f$, para lograr una mejor conservación de masa **(Flux corrector)**.

6. Se relaja la presión con un factor $\alpha_p$.

7. Se corrige la velocidad $\mathbf{u}$ previo a proceder a proceder con una nueva correción de la velocidad **(Momentum corrector)**.

8. Se vuelve a corregir la presión de nuevo, y esto es seguido de una nueva corrección de flujo ($\phi_f$) y de momentum ($\mathbf{u}$) **(PISO loop)**.

9. Se lleva a cabo una nueva iteración del algoritmo dentro de un mismo paso de tiempo. Esto permite trabajar con coeficientes actualizados dentro del paso de tiempo **(PIMPLE loop)**.

10. Los términos actualizados de presión, $p$, velocidad, $\mathbf{u}$, y flujos en las caras, $\phi_f$ seran utilizados para el paso de tiempo posterior.

<center>

<img src="figures/images/pimpleMod.png" style="width:400px" title="Geometría 1D">

***Figura 03. Esquema del algoritmo PIMPLE.***

</center>

El término de presión expresando en la figura corresponde a una "presión dinámica", $\hat{p} = p/\rho$.

Adaptado de: [[CFD Direct, 2023]](https://doc.cfd.direct/notes/cfd-general-principles/numerical-method)

### Esquemas "Fully Coupled"

De forma general, este tipo de esquemas se basan en resolver en construir una matriz masiva, que incluye los campos $\mathbf{u}$, $p$.

- Al tener un sistema de ecuaciones mas grande, el requerimiento de memoria es mayor.

- Se espera que la convergencia sea más rápida.

$$
\begin{bmatrix}
\square & * & & * \\
* & \square & * & \\
 & * & \square & * \\
* & & * & \square
\end{bmatrix}
\begin{bmatrix}
u_x \\ u_y \\ u_z\\ p \\
\end{bmatrix}
=
\begin{bmatrix}
* \\ * \\ * \\ *
\end{bmatrix}
$$